In [1]:
# WORK IN PROGRESS
from typing import Iterable

import rich

from mdmodels import DataModel, llm  # noqa

In [2]:
enzymeml = DataModel.from_github(
    repo="EnzymeML/enzymeml-specifications",
    branch="enzymeml-2",
    spec_path="specifications/enzymeml.md"
)

In [6]:
response = llm.query_openai(
    response_model=enzymeml.EnzymeMLDocument,
    pre_prompt="You are a scientist working with EnzymeML data. Think step by step about the process of creating a new EnzymeML file for a new enzyme.",
    query="Create an EnzymeML document that describes alcohol degradation by alcohol dehydrogenase, "
          "including molecules and reactions schemes. No measurements and equations needed.",
)

rich.print(response)

Output()

Response[EnzymeMLDocument](
    questions=[],
    answer='',
    chain_of_thought='To create an EnzymeML document for alcohol degradation by alcohol dehydrogenase, I need to 
define the proteins, small molecules, reactions, and vessels involved in the process. Alcohol dehydrogenase is the 
enzyme (protein) that catalyzes the conversion of alcohol (ethanol) to acetaldehyde. The reaction involves ethanol 
as the substrate and NAD+ as a cofactor, producing acetaldehyde and NADH. I will create entries for these 
components and the reaction scheme.',
    data=EnzymeMLDocument(
        name='Alcohol Degradation by Alcohol Dehydrogenase',
        references=[],
        created=None,
        modified=None,
        creators=[],
        vessels=[
            Vessel(
                id='v1',
                name='Reaction Vessel',
                volume=1.0,
                unit=UnitDefinition(
                    id='u1',
                    name='litre',
                    base_units=[
                        BaseUnit(kind=<UnitType.LITRE: 'litre'>, exponent=1, multiplier=None, scale=None)
                    ]
                ),
                constant=True
            )
        ],
        proteins=[
            Protein(
                id='p1',
                name='Alcohol Dehydrogenase',
                constant=False,
                sequence=None,
                vessel_id='v1',
                ecnumber='1.1.1.1',
                organism='Homo sapiens',
                organism_tax_id='9606',
                references=[]
            )
        ],
        complexes=[],
        small_molecules=[
            SmallMolecule(
                id='sm1',
                name='Ethanol',
                constant=False,
                vessel_id='v1',
                canonical_smiles='CCO',
                inchi='InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3',
                inchikey='LFQSCWFLJHTTHZ-UHFFFAOYSA-N',
                references=[]
            ),
            SmallMolecule(
                id='sm2',
                name='NAD+',
                constant=False,
                vessel_id='v1',
                canonical_smiles=None,
                inchi=None,
                inchikey=None,
                references=[]
            ),
            SmallMolecule(
                id='sm3',
                name='Acetaldehyde',
                constant=False,
                vessel_id='v1',
                canonical_smiles='CC=O',
                inchi='InChI=1S/C2H4O/c1-2-3/h2H,1H3',
                inchikey='IKHGUXGNUITLKF-UHFFFAOYSA-N',
                references=[]
            ),
            SmallMolecule(
                id='sm4',
                name='NADH',
                constant=False,
                vessel_id='v1',
                canonical_smiles=None,
                inchi=None,
                inchikey=None,
                references=[]
            )
        ],
        reactions=[
            Reaction(
                id='r1',
                name='Ethanol Oxidation',
                reversible=False,
                kinetic_law=None,
                species=[
                    ReactionElement(species_id='sm1', stoichiometry=1.0),
                    ReactionElement(species_id='sm2', stoichiometry=1.0),
                    ReactionElement(species_id='sm3', stoichiometry=-1.0),
                    ReactionElement(species_id='sm4', stoichiometry=-1.0)
                ],
                modifiers=['p1']
            )
        ],
        measurements=[],
        equations=[],
        parameters=[]
    )
)

In [8]:
# Now lets map the response to the STRENDA metadata model
strenda = DataModel.from_github(
    repo="Strenda-biocatalysis/Strenda-biocatalysis",
    spec_path="specifications/biocatalyst.md",
    branch="main",
)

biocatalysts = llm.query_openai(
    pre_prompt="You are tasked to map the EnzymeML document to the STRENDA metadata model provided as a response model.",
    query="Map the EnzymeML document to the STRENDA metadata model. Leave optional fields as None if not present.",
    response_model=Iterable[strenda.BiocatalystPurchased],
    previous_data_response=response,
    use_scaffold=False
)

rich.print(biocatalysts)

Output()

[
    BiocatalystPurchased(
        name='Alcohol Dehydrogenase',
        ec_number='1.1.1.1',
        molecular_weight=0.0,
        molecular_weight_unit='',
        catalyzed_reaction='Ethanol + NAD+ -> Acetaldehyde + NADH',
        sequence_amino_acid='',
        sequence_DNA='',
        origin_organism='Homo sapiens',
        supplier='',
        production_organism='',
        posttranslational_modification='',
        purity=0.0,
        purity_specification='',
        formulation=''
    )
]